In [4]:
import pandas as pd
import numpy as np

In [5]:
PATH = 'D:\\Run Analysis\\blob_storage\\truck_exp_other\\original_ds\\20211024_motorVehicle.xlsx'
COMPLETEPATH = 'D:\\Run Analysis\\blob_storage\\truck_exp_other\\transformed_og_ds\\40th_2021.csv'   
df = pd.read_excel(PATH)
df.Date = pd.to_datetime(df.Date, format="%d/%m/%Y")
df[df.Date.between('20211006', '20211012')].to_csv(COMPLETEPATH,index=False)

In [6]:
WEEK = "40th_2021"

PATH_TRANSFORMED_MVEXP = f"D:\\Run Analysis\\blob_storage\\truck_exp_other\\transformed_og_ds\\{WEEK}.csv"

PATH_ROSTER = f"D:\\Run Analysis\\blob_storage\\roster\\{WEEK}.xlsx" 

mvexp_df = pd.read_csv(PATH_TRANSFORMED_MVEXP)

roster_df = pd.read_excel(PATH_ROSTER)


In [7]:
mvexp_df_trucks_arr = mvexp_df['Job No.'].unique()

In [8]:
pri_trucks = roster_df.dropna(subset=['Primary_truck']).Primary_truck.str.replace(' ','').unique()
sec_trucks = roster_df.dropna(subset=['Alternative_Truck']).Alternative_Truck.str.replace(' ','').unique()
# pri_trucks
arr = np.append(pri_trucks, sec_trucks)
uniqArr = list(set(arr))
ros_trucks = np.array(uniqArr)


In [9]:
np.setdiff1d(ros_trucks,mvexp_df_trucks_arr  ,assume_unique=True)

array(['XN72FD', 'XN75OS', 'CG94KY', 'CH32FV', 'XN10HU', 'CJ65PS',
       'XO62AB', 'XN81VB', 'XN76VB', 'CL98LU', 'XN19IN', 'XN71FD',
       'XN65IJ', 'XN70OD', 'XN25GD', 'XN62UD', 'XN44QZ', 'CQ64LO'],
      dtype='<U6')

In [10]:
np.setdiff1d(mvexp_df_trucks_arr,ros_trucks  ,assume_unique=True)

array(['New Front Lift', 'New Hook Lift', 'FLEET', 'XN13KR'], dtype=object)

In [73]:
PATH = "D:\\Run Analysis\\blob_storage\\roster\\32th_2021.xlsx"
roster_df = pd.read_excel(PATH)


def clean_roster_df(df):
    df.Primary_truck = df.Primary_truck.str.replace(" ", "")
    df.Alternative_Truck = df.Alternative_Truck.str.replace(" ", "")
    df = df.fillna(0)
    return df

roster_df = clean_roster_df(roster_df)


pri_truck_runs_count = roster_df.Primary_truck.value_counts()
sec_truck_runs_count = roster_df.Alternative_Truck.value_counts()

# =============================================================================================
# Counting Trucks runs
total_truck_runs_count = (pd
                            .concat([pri_truck_runs_count, sec_truck_runs_count])
                            .reset_index()
                            .groupby('index')
                            .sum().iloc[1:]
                            .reset_index()
                            .rename(columns={"index" : "number_plate", 0: "runs"})
                            )
trucks_cost_df = pd.merge(mvexp_df, total_truck_runs_count, how="left", left_on='Job No.', right_on='number_plate')
trucks_cost_df['per_trucks_cost']= trucks_cost_df.Debit / trucks_cost_df.runs

In [80]:
trucks_cost_df = trucks_cost_df[[
        'Acc_name', 'Job No.', 'per_trucks_cost']].dropna(subset=['per_trucks_cost'])

pri_trucks_cost = pd.merge(roster_df, trucks_cost_df, how="left",
                           left_on='Primary_truck', right_on='Job No.').dropna(subset=['per_trucks_cost'])

alt_trucks_cost = pd.merge(roster_df, trucks_cost_df, how="left",
                           left_on='Alternative_Truck', right_on='Job No.').dropna(subset=['per_trucks_cost'])

trucks_cost_summary = pd.concat([pri_trucks_cost, alt_trucks_cost])

In [81]:
trucks_cost_summary
# pd.merge(roster_df, trucks_cost_df, how='left', left_on='') Job No.

,Date,Run_type,Primary_employeeID,Secondary_employeeID,Primary_driver_Name,Secondary_Driver_Name,Primary_route,Satellite_Route_1,Satellite_Route_2,Primary_truck,Alternative_Truck,Subcontracted_From/Special_Client,Note,Acc_name,Job No.,per_trucks_cost
7,2021-08-13,NIGHT RUNS,subcon1,0,ALAN WALDEN,0,RLR,RED,0,XPL658,0,0,0.0,MV R&M - Cab&Chassic,XPL658,25.000000
8,2021-08-13,NIGHT RUNS,subcon1,0,ALAN WALDEN,0,RLR,RED,0,XPL658,0,0,0.0,MV R&M - Compactor/Body,XPL658,17.931667
21,2021-08-11,DAY RUNS,swedwben,0,BEN EDWARDS,0,SWG,0,0,XN72MK,0,0,0.0,MV R&M - Cab&Chassic,XN72MK,30.000000
22,2021-08-11,DAY RUNS,swedwben,0,BEN EDWARDS,0,SWG,0,0,XN72MK,0,0,0.0,M.V R&M - Tyres,XN72MK,9.000000
23,2021-08-12,DAY RUNS,swedwben,0,BEN EDWARDS,0,SWG,0,0,XN72MK,0,0,0.0,MV R&M - Cab&Chassic,XN72MK,30.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021-08-11,NIGHT RUNS,swnevpj,ir20,PAUL,TOM,BR2,RL9,0,XN80YD,BL82AG,0,0.0,MV R&M - Cab&Chassic,BL82AG,37.500000
96,2021-08-12,NIGHT RUNS,swnevpj,ir20,PAUL,TOM,BR2,RL9,0,CQ64LO,XN80YD,0,0.0,MV R&M - Cab&Chassic,XN80YD,5.311538
97,2021-08-12,DAY RUNS,swnevpj,ir20,PAUL,TOM,BR2,RL9,0,XN80YD,BL82AG,0,0.0,MV R&M - Cab&Chassic,BL82AG,37.500000
99,2021-08-16,NIGHT RUNS,swnevpj,ir20,PAUL,TOM,BR2,RL9,0,XN80YD,BL82AG,0,0.0,MV R&M - Cab&Chassic,BL82AG,37.500000


In [78]:
roster_df.head(3)

,Date,Run_type,Primary_employeeID,Secondary_employeeID,Primary_driver_Name,Secondary_Driver_Name,Primary_route,Satellite_Route_1,Satellite_Route_2,Primary_truck,Alternative_Truck,Subcontracted_From/Special_Client,Note
0,2021-08-11,NIGHT RUNS,swaklado,0,ADONIS AKL,0,RLE,0,0,CL98LU,0,0,0.0
1,2021-08-12,NIGHT RUNS,swaklado,0,ADONIS AKL,0,RLE,0,0,CL98LU,0,0,0.0
2,2021-08-13,NIGHT RUNS,swaklado,0,ADONIS AKL,0,RLE,0,0,CL98LU,0,0,0.0


In [79]:
trucks_cost_df.head(3)

,Acc_name,Job No.,per_trucks_cost
0,MV R&M - Cab&Chassic,BY46TO,19.856000
1,MV R&M - Cab&Chassic,XN80YD,5.311538
3,MV R&M - Cab&Chassic,XPL658,25.000000


In [ ]:


df_100 = df_99[['Acc_name', 'Job No.', 'Debit','per_run_cost']].dropna(subset=['per_run_cost'])

pri_truck_cost = pd.merge(roster_df, df_100, how='left', left_on='Primary_truck', right_on='Job No.')

p_df = pri_truck_cost[['Date', 'Primary_route', 'Satellite_Route_1', 'Satellite_Route_2', 'per_run_cost']].dropna(subset=['per_run_cost'])
alt_truck_cost = pd.merge(roster_df, df_100, how='left', left_on='Alternative_Truck', right_on='Job No.')
alt_truck_cost = alt_truck_cost.dropna(subset=['per_run_cost'])

a_df = alt_truck_cost[['Date', 'Primary_route', 'Satellite_Route_1', 'Satellite_Route_2', 'per_run_cost']].dropna(subset=['per_run_cost'])
t1 = pd.concat([p_df, a_df])
# t1[t1.Satellite_Route_2.ne(0)].set_index(['Date', 'per_run_cost']) .apply(lambda x: x['Satellite_Route_2'].split('/') if type(x['Satellite_Route_2']) == str else 0, axis=1).explode().reset_index()
t2 = (t1[t1.Satellite_Route_2.ne(0)]
    .set_index(['Date', 'per_run_cost'])
        .apply(lambda x: x['Satellite_Route_2']
        .split('/') if type(x['Satellite_Route_2']) == str else 0, axis=1)
        .explode()
        .reset_index()
        .rename(columns={ 0 : 'route'}))
t2